In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import time

In [2]:
#Loading the webpage

url = 'https://stats.espncricinfo.com/ci/engine/stats/index.html?class=3;template=results;type=batting'
header = {'User Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'}
r = requests.get(url)
soup = BeautifulSoup(r.content,'lxml')
#print(soup.prettify())

In [3]:
#Selecting the data table from the webpage

Player_table = soup.find_all("table",class_ = 'engineTable')[2]
print(Player_table)

<table class="engineTable">
<caption>Overall figures</caption>
<thead>
<tr class="headlinks">
<th class="left" nowrap=""><a class="black-link" href="/ci/engine/stats/index.html?class=3;orderby=player;template=results;type=batting" title="sort by player name">Player</a></th>
<th class="left" nowrap=""><a class="black-link" href="/ci/engine/stats/index.html?class=3;orderby=start;template=results;type=batting" title="sort by start date">Span</a></th>
<th nowrap=""><a class="black-link" href="/ci/engine/stats/index.html?class=3;orderby=matches;template=results;type=batting" title="sort by matches played">Mat</a></th>
<th nowrap=""><a class="black-link" href="/ci/engine/stats/index.html?class=3;orderby=innings;template=results;type=batting" title="sort by innings batted">Inns</a></th>
<th nowrap=""><a class="black-link" href="/ci/engine/stats/index.html?class=3;orderby=notouts;template=results;type=batting" title="sort by not outs">NO</a></th>
<th nowrap=""><a class="black-link" href="/ci/e

In [4]:
#Creating a template to extract data from the table in webpage

for name in Player_table.find_all('tbody'):
    rows = name.find_all('tr')
    for row in rows:
        player = row.find('td',class_ = 'left').text
        span = row.find_all('td')[1].text
        mat = row.find_all('td')[2].text
        inns = row.find_all('td')[3].text
        NO = row.find_all('td')[4].text
        runs = row.find_all('td')[5].text
        HS = row.find_all('td')[6].text
        Ave = row.find_all('td')[7].text
        BF = row.find_all('td')[8].text
        SR = row.find_all('td')[9].text
        Hundreds = row.find_all('td')[10].text
        Fifties = row.find_all('td')[11].text
        Zero = row.find_all('td')[12].text
        Fours = row.find_all('td')[13].text
        Sixes = row.find_all('td')[14].text
        #Storing it in a dictionary
        Cricketers = {'Player':player,"Span":span,"Matches":mat,"Innings":inns,"NO":NO,"Runs":runs,"HS":HS,
                         "AVG":Ave,"Balls Faced":BF,"Strike Rate":SR,"100":Hundreds,"50":Fifties,"0":Zero,"4's":Fours,
                         "6's":Sixes}
        #print(Cricketers)


In [5]:
#Assigning an empty list to store all dictionaries of players

Cricketers_List = []

#Automating scrapping through different pages[the code has only first 2 page loop] by using a for loop and an f string edit in the link.

for page in range(1,56):
    source = requests.get(f'https://stats.espncricinfo.com/ci/engine/stats/index.html?class=3;page={page};template=results;type=batting')
    soup = BeautifulSoup(source.content,'lxml')
    Player_table = soup.find_all("table",class_ = 'engineTable')[2]
    
    #Code from the above cell which builds table structure
    
    for name in Player_table.find_all('tbody'):
        rows = name.find_all('tr')
        
        if len(name)%10 == 0:
            print(len(name))
            
        for row in rows:
            player = row.find('td',class_ = 'left').text
            span = row.find_all('td')[1].text
            mat = row.find_all('td')[2].text
            inns = row.find_all('td')[3].text
            NO = row.find_all('td')[4].text
            runs = row.find_all('td')[5].text
            HS = row.find_all('td')[6].text
            Ave = row.find_all('td')[7].text
            BF = row.find_all('td')[8].text
            SR = row.find_all('td')[9].text
            Hundreds = row.find_all('td')[10].text
            Fifties = row.find_all('td')[11].text
            Zero = row.find_all('td')[12].text
            Fours = row.find_all('td')[13].text
            Sixes = row.find_all('td')[14].text
            
            Cricketers = {'Player':player,"Span":span,"Matches":mat,"Innings":inns,"NO":NO,"Runs":runs,"HS":HS,
                         "AVG":Ave,"Balls Faced":BF,"Strike Rate":SR,"100":Hundreds,"50":Fifties,"0":Zero,"4's":Fours,
                         "6's":Sixes}
            Cricketers_List.append(Cricketers)

In [6]:
#Converting to DataFrame using pandas [sorted by number of runs by default]

dft20batting = pd.DataFrame(Cricketers_List)
dft20batting

,Player,Span,Matches,Innings,NO,Runs,HS,AVG,Balls Faced,Strike Rate,100,50,0,4's,6's
0,RG Sharma (INDIA),2007-2022,125,117,15,3313,118,32.48,2374,139.55,4,26,7,293,155
1,MJ Guptill (NZ),2009-2021,112,108,7,3299,105,32.66,2413,136.71,2,20,3,287,165
2,V Kohli (INDIA),2010-2022,97,89,25,3296,94*,51.50,2394,137.67,0,30,3,298,92
3,PR Stirling (IRE),2009-2022,102,101,8,2776,115*,29.84,2061,134.69,1,20,9,319,96
4,AJ Finch (AUS),2011-2022,89,89,10,2741,172,34.69,1892,144.87,2,16,7,273,113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2708,Zamir Khan (AFG),2012-2012,1,-,-,-,-,-,-,-,-,-,-,-,-
2709,S Zargar (Mex),2019-2019,1,-,-,-,-,-,-,-,-,-,-,-,-
2710,Zeshan Arif (Aut),2019-2019,3,-,-,-,-,-,-,-,-,-,-,-,-
2711,Ziaur Rahman (AFG),2019-2019,1,-,-,-,-,-,-,-,-,-,-,-,-


In [8]:
#Saving it into CSV

dft20batting.to_csv("T20 Batsmen.csv")

In [14]:
dft20batting.set_axis(['Player','T20_Span','T20_Matches','T20_Innings','T20_Notouts','T20_RunsMade','T20_HighestScore','T20_Batting_Average','T20_BallsFaced','T20_Batting_Strikerate','T20_Hundereds','T20_Fifties','T20_Ducks','T20_Fours','T20_Sixes'],axis=1,inplace=True)

In [10]:
dft20batting=dft20batting.replace('-', 0)

In [13]:
dft20batting

,Player,T20_Span,T20_Matches,T20_Innings,T20_Notouts,T20_RunsMade,T20_HighestScore,T20_Batting_Average,T20_BallsFaced,T20_Strikerate,T20_Hundereds,T20_Fifties,T20_Ducks,T20_Fours,T20_Sixes
0,RG Sharma (INDIA),2007-2022,125,117,15,3313,118,32.48,2374,139.55,4,26,7,293,155
1,MJ Guptill (NZ),2009-2021,112,108,7,3299,105,32.66,2413,136.71,2,20,3,287,165
2,V Kohli (INDIA),2010-2022,97,89,25,3296,94*,51.50,2394,137.67,0,30,3,298,92
3,PR Stirling (IRE),2009-2022,102,101,8,2776,115*,29.84,2061,134.69,1,20,9,319,96
4,AJ Finch (AUS),2011-2022,89,89,10,2741,172,34.69,1892,144.87,2,16,7,273,113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2708,Zamir Khan (AFG),2012-2012,1,0,0,0,0,0,0,0,0,0,0,0,0
2709,S Zargar (Mex),2019-2019,1,0,0,0,0,0,0,0,0,0,0,0,0
2710,Zeshan Arif (Aut),2019-2019,3,0,0,0,0,0,0,0,0,0,0,0,0
2711,Ziaur Rahman (AFG),2019-2019,1,0,0,0,0,0,0,0,0,0,0,0,0


In [16]:
#'Player','T20_Span','T20_Matches','T20_Innings','T20_Notouts','T20_RunsMade','T20_HighestScore','T20_Batting_Average','T20_BallsFaced','T20_Strikerate','T20_Hundereds','T20_Fifties','T20_Ducks','T20_Fours','T20_Sixes'

dft20batting['T20_Matches']=dft20batting['T20_Matches'].astype(int)
dft20batting['T20_Innings']=dft20batting['T20_Innings'].astype(int)
dft20batting['T20_Notouts']=dft20batting['T20_Notouts'].astype(int)
dft20batting['T20_RunsMade']=dft20batting['T20_RunsMade'].astype(int)
dft20batting['T20_BallsFaced']=dft20batting['T20_BallsFaced'].astype(int)
dft20batting['T20_Hundereds']=dft20batting['T20_Hundereds'].astype(int)
dft20batting['T20_Fifties']=dft20batting['T20_Fifties'].astype(int)
dft20batting['T20_Ducks']=dft20batting['T20_Ducks'].astype(int)
dft20batting['T20_Fours']=dft20batting['T20_Fours'].astype(int)
dft20batting['T20_Sixes']=dft20batting['T20_Sixes'].astype(int)
dft20batting["T20_Batting_Average"] = pd.to_numeric(dft20batting['T20_Batting_Average'], errors='coerce')
dft20batting["T20_Batting_Strikerate"] = pd.to_numeric(dft20batting['T20_Batting_Strikerate'], errors='coerce')
dft20batting.dtypes


Player                     object
T20_Span                   object
T20_Matches                 int32
T20_Innings                 int32
T20_Notouts                 int32
T20_RunsMade                int32
T20_HighestScore           object
T20_Batting_Average       float64
T20_BallsFaced              int32
T20_Batting_Strikerate    float64
T20_Hundereds               int32
T20_Fifties                 int32
T20_Ducks                   int32
T20_Fours                   int32
T20_Sixes                   int32
dtype: object

In [17]:
dft20batting

,Player,T20_Span,T20_Matches,T20_Innings,T20_Notouts,T20_RunsMade,T20_HighestScore,T20_Batting_Average,T20_BallsFaced,T20_Batting_Strikerate,T20_Hundereds,T20_Fifties,T20_Ducks,T20_Fours,T20_Sixes
0,RG Sharma (INDIA),2007-2022,125,117,15,3313,118,32.48,2374,139.55,4,26,7,293,155
1,MJ Guptill (NZ),2009-2021,112,108,7,3299,105,32.66,2413,136.71,2,20,3,287,165
2,V Kohli (INDIA),2010-2022,97,89,25,3296,94*,51.50,2394,137.67,0,30,3,298,92
3,PR Stirling (IRE),2009-2022,102,101,8,2776,115*,29.84,2061,134.69,1,20,9,319,96
4,AJ Finch (AUS),2011-2022,89,89,10,2741,172,34.69,1892,144.87,2,16,7,273,113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2708,Zamir Khan (AFG),2012-2012,1,0,0,0,0,0.00,0,0.00,0,0,0,0,0
2709,S Zargar (Mex),2019-2019,1,0,0,0,0,0.00,0,0.00,0,0,0,0,0
2710,Zeshan Arif (Aut),2019-2019,3,0,0,0,0,0.00,0,0.00,0,0,0,0,0
2711,Ziaur Rahman (AFG),2019-2019,1,0,0,0,0,0.00,0,0.00,0,0,0,0,0


In [18]:
dft20batting.to_excel('T20_Batting.xlsx')

In [2]:
df_t02= pd.read_excel('T20_Batting.xlsx')
df_t02


,Unnamed: 0,Player,T20_Span,T20_Matches,T20_Innings,T20_Notouts,T20_RunsMade,T20_HighestScore,T20_Batting_Average,T20_BallsFaced,T20_Batting_Strikerate,T20_Hundereds,T20_Fifties,T20_Ducks,T20_Fours,T20_Sixes
0,0,RG Sharma (INDIA),2007-2022,125,117,15,3313,118,32.48,2374,139.55,4,26,7,293,155
1,1,MJ Guptill (NZ),2009-2021,112,108,7,3299,105,32.66,2413,136.71,2,20,3,287,165
2,2,V Kohli (INDIA),2010-2022,97,89,25,3296,94*,51.50,2394,137.67,0,30,3,298,92
3,3,PR Stirling (IRE),2009-2022,102,101,8,2776,115*,29.84,2061,134.69,1,20,9,319,96
4,4,AJ Finch (AUS),2011-2022,89,89,10,2741,172,34.69,1892,144.87,2,16,7,273,113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2708,2708,Zamir Khan (AFG),2012-2012,1,0,0,0,0,0.00,0,0.00,0,0,0,0,0
2709,2709,S Zargar (Mex),2019-2019,1,0,0,0,0,0.00,0,0.00,0,0,0,0,0
2710,2710,Zeshan Arif (Aut),2019-2019,3,0,0,0,0,0.00,0,0.00,0,0,0,0,0
2711,2711,Ziaur Rahman (AFG),2019-2019,1,0,0,0,0,0.00,0,0.00,0,0,0,0,0


In [4]:
#dropping the unnamed index coming
df_t02=df_t02.drop(['Unnamed: 0'],axis=1)
df_t02

,Player,T20_Span,T20_Matches,T20_Innings,T20_Notouts,T20_RunsMade,T20_HighestScore,T20_Batting_Average,T20_BallsFaced,T20_Batting_Strikerate,T20_Hundereds,T20_Fifties,T20_Ducks,T20_Fours,T20_Sixes
0,RG Sharma (INDIA),2007-2022,125,117,15,3313,118,32.48,2374,139.55,4,26,7,293,155
1,MJ Guptill (NZ),2009-2021,112,108,7,3299,105,32.66,2413,136.71,2,20,3,287,165
2,V Kohli (INDIA),2010-2022,97,89,25,3296,94*,51.50,2394,137.67,0,30,3,298,92
3,PR Stirling (IRE),2009-2022,102,101,8,2776,115*,29.84,2061,134.69,1,20,9,319,96
4,AJ Finch (AUS),2011-2022,89,89,10,2741,172,34.69,1892,144.87,2,16,7,273,113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2708,Zamir Khan (AFG),2012-2012,1,0,0,0,0,0.00,0,0.00,0,0,0,0,0
2709,S Zargar (Mex),2019-2019,1,0,0,0,0,0.00,0,0.00,0,0,0,0,0
2710,Zeshan Arif (Aut),2019-2019,3,0,0,0,0,0.00,0,0.00,0,0,0,0,0
2711,Ziaur Rahman (AFG),2019-2019,1,0,0,0,0,0.00,0,0.00,0,0,0,0,0


In [5]:
#As checked we are finding 2 major errors in player column.. 1st is some player are having name+(numericvalue)+(countryname)
#2nd having ICC/World/Asia/Afr along with country name 

#1st removing (numeric data)
df_t02['Player']=df_t02['Player'].str.replace(r"\(.1?\)","")
df_t02['Player']=df_t02['Player'].str.replace(r"\(.2?\)","")
df_t02['Player']=df_t02['Player'].str.replace(r"\(.3?\)","")

<ipython-input-5-6ecb8ace70e4>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df_t02['Player']=df_t02['Player'].str.replace(r"\(.1?\)","")
<ipython-input-5-6ecb8ace70e4>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df_t02['Player']=df_t02['Player'].str.replace(r"\(.2?\)","")
<ipython-input-5-6ecb8ace70e4>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df_t02['Player']=df_t02['Player'].str.replace(r"\(.3?\)","")


In [6]:
#2nd replacing ICC/World/Asia/Afr from the country name

df_t02= df_t02.replace('ICC/', '', regex=True)
df_t02= df_t02.replace('/ICC', '', regex=True)
df_t02= df_t02.replace('/Afr', '', regex=True)
df_t02= df_t02.replace('Afr/', '', regex=True)
df_t02= df_t02.replace('/Asia', '', regex=True)  
df_t02= df_t02.replace('Asia/', '', regex=True)  
df_t02= df_t02.replace('/World', '', regex=True)    


In [7]:
df_t02

,Player,T20_Span,T20_Matches,T20_Innings,T20_Notouts,T20_RunsMade,T20_HighestScore,T20_Batting_Average,T20_BallsFaced,T20_Batting_Strikerate,T20_Hundereds,T20_Fifties,T20_Ducks,T20_Fours,T20_Sixes
0,RG Sharma (INDIA),2007-2022,125,117,15,3313,118,32.48,2374,139.55,4,26,7,293,155
1,MJ Guptill (NZ),2009-2021,112,108,7,3299,105,32.66,2413,136.71,2,20,3,287,165
2,V Kohli (INDIA),2010-2022,97,89,25,3296,94*,51.50,2394,137.67,0,30,3,298,92
3,PR Stirling (IRE),2009-2022,102,101,8,2776,115*,29.84,2061,134.69,1,20,9,319,96
4,AJ Finch (AUS),2011-2022,89,89,10,2741,172,34.69,1892,144.87,2,16,7,273,113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2708,Zamir Khan (AFG),2012-2012,1,0,0,0,0,0.00,0,0.00,0,0,0,0,0
2709,S Zargar (Mex),2019-2019,1,0,0,0,0,0.00,0,0.00,0,0,0,0,0
2710,Zeshan Arif (Aut),2019-2019,3,0,0,0,0,0.00,0,0.00,0,0,0,0,0
2711,Ziaur Rahman (AFG),2019-2019,1,0,0,0,0,0.00,0,0.00,0,0,0,0,0


In [11]:
df_t02.to_excel('T20_Battingv3.xlsx')